## Projeto Analise Feedback

Este notebook tem como objetivo desenvolver dois modelos distintos de classificação de texto usando técnicas clássicas de Processamento de Linguagem Natural (NLP):

1. **Classificação de Sentimentos**: Dado um conjunto de avaliações de produtos, o modelo identifica se cada avaliação é **positiva** ou **negativa**.
2. **Classificação de Sugestões de Melhoria**: Dado um conjunto de frases variadas, o modelo identifica se o texto representa ou não uma **sugestão de melhoria**.


### 📦 Importação das Bibliotecas
Importamos as bibliotecas necessárias para manipulação de dados, treinamento de modelos e avaliação de desempenho.

In [21]:
import pandas as pd
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

### 📥 Carregamento dos Dados
Aqui carregamos dois conjuntos de dados:

- Sugestões de melhoria (sugestoes.txt)  
      Fonte: Gerada por multiplas IA 
- Avaliações de produtos (B2W-Reviews01.csv)  
      Fonte: https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets

In [22]:
print("--- Iniciando o carregamento dos dados ---")

# --- 1.1 Carregando as SUGESTÕES (melhorias.txt) ---
arquivo_sugestoes = '../data/sugestoes.txt'
try:
    with open(arquivo_sugestoes, 'r', encoding='latin1') as f:
        linhas = [linha.strip() for linha in f.readlines()]
    df_sugestoes = pd.DataFrame(linhas, columns=['review_text'])
    df_sugestoes['is_suggestion'] = 1
    print(f"✅ Dataset de SUGESTÕES carregado com {len(df_sugestoes)} exemplos.")
except Exception as e:
    print(f"❌ ERRO ao carregar o arquivo de sugestões: {e}")

# --- 1.2 Carregando as OPINIÕES (B2W-Reviews01.csv) ---
arquivo_opinioes = '../data/B2W-Reviews01.csv'
try:
    df_b2w = pd.read_csv(arquivo_opinioes, sep=',', dtype={'product_id': str})
    df_b2w.dropna(subset=['review_text'], inplace=True)
    print(f"✅ Dataset de OPINIÕES (B2W) carregado com {len(df_b2w)} exemplos.")
except Exception as e:
    print(f"❌ ERRO ao carregar o arquivo B2W: {e}")

--- Iniciando o carregamento dos dados ---
✅ Dataset de SUGESTÕES carregado com 1506 exemplos.
✅ Dataset de OPINIÕES (B2W) carregado com 129098 exemplos.


### 🧹 Preparação dos Dados
Preparamos os dados para dois objetivos diferentes:

In [23]:
print("\n--- Preparando os dados para cada modelo de treinamento ---")

# --- 2.1 Preparando os dados para o MODELO DE SENTIMENTO ---
# Este modelo usa APENAS o dataset B2W para aprender o que é positivo e negativo.

# Cria um novo DataFrame a partir do B2W
df_para_sentimento = df_b2w[['review_text', 'overall_rating']].copy()

# Função para mapear o sentimento
def map_sentiment(rating):
    if rating <= 2: return 0  # Negativo
    elif rating >= 4: return 1  # Positivo
    else: return None

# Aplica o mapeamento e remove as avaliações neutras
df_para_sentimento['sentiment'] = df_para_sentimento['overall_rating'].apply(map_sentiment)
df_para_sentimento.dropna(subset=['sentiment'], inplace=True)
df_para_sentimento['sentiment'] = df_para_sentimento['sentiment'].astype(int)

# Define X e y para este modelo
X_sentimento = df_para_sentimento['review_text']
y_sentimento = df_para_sentimento['sentiment']

print("\n✅ Dados para o Modelo de Sentimento prontos.")
print(y_sentimento.value_counts())

# --- 2.2 Preparando os dados para o MODELO DE SUGESTÃO ---
# Este modelo usa as sugestões do seu arquivo .txt (classe 1) e
# uma amostra de opiniões "puras" do B2W (classe 0).

# Define palavras-chave para FILTRAR o dataset B2W e pegar apenas opiniões "puras"
suggestion_keywords = [
    'sugiro', 'sugestão', 'sugestões', 'poderia', 'poderiam', 
    'deveria', 'deviam', 'recomendo que', 'adicionar', 
    'melhorar', 'implementar', 'faltou', 'seria bom se'
]
keyword_pattern = '|'.join(suggestion_keywords)

# Filtra o B2W, mantendo apenas as linhas que NÃO contêm as palavras-chave, e cria uma cópia
df_opinioes_limpas = df_b2w[~df_b2w['review_text'].str.contains(keyword_pattern, case=False, na=False)].copy()
df_opinioes_limpas['is_suggestion'] = 0

# Cria um dataset balanceado
# Pega uma amostra das opiniões limpas com o mesmo tamanho do seu dataset de sugestões
df_opinioes_sample = df_opinioes_limpas.sample(n=len(df_sugestoes), random_state=42)

# Combina as sugestões (classe 1) com a amostra de opiniões (classe 0)
df_para_sugestao = pd.concat([df_sugestoes, df_opinioes_sample[['review_text', 'is_suggestion']]], ignore_index=True)
df_para_sugestao = df_para_sugestao.sample(frac=1, random_state=42).reset_index(drop=True) # Embaralha

# Define X e y para este segundo modelo
X_sugestao = df_para_sugestao['review_text']
y_sugestao = df_para_sugestao['is_suggestion']

print("\n✅ Dados para o Modelo de Sugestão prontos (dataset balanceado).")
print(y_sugestao.value_counts())


--- Preparando os dados para cada modelo de treinamento ---

✅ Dados para o Modelo de Sentimento prontos.
sentiment
1    79316
0    33772
Name: count, dtype: int64

✅ Dados para o Modelo de Sugestão prontos (dataset balanceado).
is_suggestion
1    1506
0    1506
Name: count, dtype: int64


### 🤖 Treinamento do Modelo de Sentimento
Utilizamos um Pipeline com TF-IDF + Regressão Logística para classificar os sentimentos nas avaliações.

In [27]:
print("\n--- Treinando o Modelo de Sentimento ---")

# 1. Dividir os dados de SENTIMENTO em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_sentimento, y_sentimento, test_size=0.2, random_state=42, stratify=y_sentimento
)

# 2. Criar o pipeline para o modelo de sentimento
pipeline_sentimento = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=50000)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# 3. Treinar o pipeline
print("Iniciando o treinamento do pipeline de sentimento...")
pipeline_sentimento.fit(X_train, y_train)
print("Treinamento concluído!")

# 4. Avaliar o modelo
print("\nAvaliando o modelo de sentimento...")
y_pred = pipeline_sentimento.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

# 5. Salvar o pipeline treinado
MODELS_DIR = '../models'
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

joblib.dump(pipeline_sentimento, os.path.join(MODELS_DIR, 'sentiment_pipeline.pkl'))
print(f"✅ Pipeline de Sentimento salvo em '{os.path.join(MODELS_DIR, 'sentiment_pipeline.pkl')}'")


--- Treinando o Modelo de Sentimento ---
Iniciando o treinamento do pipeline de sentimento...
Treinamento concluído!

Avaliando o modelo de sentimento...
              precision    recall  f1-score   support

    Negativo       0.93      0.91      0.92      6755
    Positivo       0.96      0.97      0.97     15863

    accuracy                           0.95     22618
   macro avg       0.94      0.94      0.94     22618
weighted avg       0.95      0.95      0.95     22618

✅ Pipeline de Sentimento salvo em '../models\sentiment_pipeline.pkl'


### 🧠 Treinamento do Modelo de Sugestão
Mesma arquitetura, mas focada em identificar sugestões explícitas dentro de textos.

In [25]:
print("\n\n--- Treinando o Modelo de Sugestão ---")

# 1. Dividir os dados de SUGESTÃO em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_sugestao, y_sugestao, test_size=0.2, random_state=42, stratify=y_sugestao
)

# 2. Criar o pipeline para o modelo de sugestão
# Podemos usar a mesma arquitetura de modelo
pipeline_sugestao = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=50000)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# 3. Treinar o pipeline
print("Iniciando o treinamento do pipeline de sugestão...")
pipeline_sugestao.fit(X_train, y_train)
print("Treinamento concluído!")

# 4. Avaliar o modelo
print("\nAvaliando o modelo de sugestão...")
y_pred = pipeline_sugestao.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Não-Sugestão', 'Sugestão']))

# 5. Salvar o segundo pipeline treinado
joblib.dump(pipeline_sugestao, os.path.join(MODELS_DIR, 'suggestion_pipeline.pkl'))
print(f"✅ Pipeline de Sugestão salvo em '{os.path.join(MODELS_DIR, 'suggestion_pipeline.pkl')}'")



--- Treinando o Modelo de Sugestão ---
Iniciando o treinamento do pipeline de sugestão...
Treinamento concluído!

Avaliando o modelo de sugestão...
              precision    recall  f1-score   support

Não-Sugestão       0.97      0.99      0.98       302
    Sugestão       0.99      0.97      0.98       301

    accuracy                           0.98       603
   macro avg       0.98      0.98      0.98       603
weighted avg       0.98      0.98      0.98       603

✅ Pipeline de Sugestão salvo em '../models\suggestion_pipeline.pkl'


### ✅ Conclusão
Neste projeto, criamos dois modelos distintos:

- Classificação de Sentimentos: Detecta se uma avaliação é positiva ou negativa com alta precisão.

- Classificação de Sugestões: Identifica se um texto contém uma sugestão ou não, sendo útil para extração de feedbacks construtivos.

Ambos os modelos demonstraram alto desempenho nos testes, com acurácia acima de 95%, o que os torna prontos para aplicações reais.